In [2]:
import pandas as pd
import numpy as np

# Target Encoding

In [ ]:
from category_encoders import TargetEncoder

# Применение target encoding
categorical_features = ['этаж расположения', 'состояние отделки', 'район расположения', 'статус ЖК', 'расстояние до метро']
encoder = TargetEncoder()
df_encoded = df.copy()
df_encoded[categorical_features] = encoder.fit_transform(df[categorical_features], df['цена'])
print(df_encoded)

# Embeddings

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Flatten
from tensorflow.keras.models import Model

# Пример категориального признака
num_categories = len(df['район расположения'].unique())
embedding_dim = 5  # Размерность вектора

# Вход для района
input_layer = Input(shape=(1,))
embedding_layer = Embedding(input_dim=num_categories, output_dim=embedding_dim, input_length=1)(input_layer)
flatten_layer = Flatten()(embedding_layer)

# Создание модели
model = Model(inputs=input_layer, outputs=flatten_layer)
model.compile(optimizer='adam', loss='mse')

# Кодирование категорий
encoded_values = model.predict(df['район расположения'])
print(encoded_values)

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Создаем DataFrame с районами
data = pd.DataFrame({
    'district': ['Центр', 'Южный', 'Северный', 'Западный', 'Восточный', 'Центр', 'Южный', 'Северный']
})

# Присваиваем индекс для районов (НЕ LabelEncoder!)
unique_districts = data['district'].unique()
district_to_idx = {district: idx for idx, district in enumerate(unique_districts)}
data['district_encoded'] = data['district'].map(district_to_idx)

# Подготовка входных данных
X_train, X_test = train_test_split(data['district_encoded'], test_size=0.2, random_state=42)

# Определяем параметры embeddings
num_districts = len(unique_districts)  # Количество районов
embedding_dim = 3  # Размерность вектора embeddings

# Определяем входные данные
input_district = keras.layers.Input(shape=(1,))
district_embedding = keras.layers.Embedding(input_dim=num_districts, output_dim=embedding_dim)(input_district)
district_embedding = keras.layers.Flatten()(district_embedding)

# Выходной слой (здесь просто обучаем embeddings, без предсказания)
output = keras.layers.Dense(embedding_dim, activation='linear')(district_embedding)

# Создаем модель
model = keras.Model(inputs=input_district, outputs=output)
model.compile(optimizer='adam', loss='mse')

# Обучаем embeddings
model.fit(X_train, X_train, epochs=100, batch_size=4, verbose=1)

# Извлекаем обученные embeddings
embedding_weights = model.get_layer(index=1).get_weights()[0]

# Создаем DataFrame с embeddings
district_embeddings = pd.DataFrame(embedding_weights, columns=[f'emb_{i}' for i in range(embedding_dim)])
district_embeddings['district'] = unique_districts

print("\nEmbeddings районов:")
print(district_embeddings)

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 5.7772 
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 5.7643
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 4.9173
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 5.4970
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 6.4742
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 5.4743
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 5.4633
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 5.4522
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 5.1730
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 6.1690
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 6.3950
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 6.3821
Epoch 13/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.8106
Epoch 14/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 4.3818
Epoch 15/100
2/2 ━

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, concatenate
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Создаем искусственные данные (как раньше)
np.random.seed(42)
n_samples = 100

area = np.random.uniform(30, 150, n_samples)  # Площадь квартиры
floor = np.random.randint(1, 20, n_samples)   # Этаж
total_floors = np.random.randint(floor, floor + 10, n_samples)  # Этажность здания
year_built = np.random.randint(1950, 2020, n_samples)  # Год постройки
condition = np.random.choice(['good', 'average', 'bad'], n_samples)  # Состояние отделки

condition_numeric = condition.map({'good': 3, 'average': 2, 'bad': 1})

price = (
    area * 1000 +
    floor * 500 +
    total_floors * 100 +
    (2020 - year_built) * 300 +
    condition_numeric * 1000
)

data = pd.DataFrame({
    'area': area,
    'floor': floor,
    'total_floors': total_floors,
    'year_built': year_built,
    'condition': condition_numeric,
    'price': price
})

# Добавляем информацию о районе
districts = np.random.choice(['District1', 'District2', 'District3', 'District4', 'District5'], n_samples)  # Пример 5 районов
data['district'] = districts

# Разделение на признаки и целевую переменную
X = data.drop('price', axis=1)
y = data['price']

# Преобразуем категориальные признаки в числовые (one-hot encoding)
X_encoded = {
    'area': X['area'].values.reshape(-1, 1),
    'floor': X['floor'].values.reshape(-1, 1),
    'total_floors': X['total_floors'].values.reshape(-1, 1),
    'year_built': X['year_built'].values.reshape(-1, 1),
    'condition': X['condition'].values.reshape(-1, 1),
    'district': pd.get_dummies(data['district']).values
}

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Создаем модель с эмбеддингами
def create_model(input_dim, embedding_dim):
    # Входные слои
    area_input = Input(shape=(1,), name='area')
    floor_input = Input(shape=(1,), name='floor')
    total_floors_input = Input(shape=(1,), name='total_floors')
    year_built_input = Input(shape=(1,), name='year_built')
    condition_input = Input(shape=(1,), name='condition')
    district_input = Input(shape=(1,), name='district')

    # Эмбеддинги для категориальных признаков
    district_embedding = Embedding(input_dim=input_dim, output_dim=embedding_dim)(district_input)
    district_embedding = tf.squeeze(district_embedding, axis=1)

    # Свертка эмбеддингов
    district_dense = Dense(10, activation='relu')(district_embedding)

    # Связывание всех входов
    concatenated = concatenate([area_input, floor_input, total_floors_input, year_built_input, condition_input, district_dense])

    # Полносвязные слои
    dense1 = Dense(64, activation='relu')(concatenated)
    dense2 = Dense(32, activation='relu')(dense1)
    output = Dense(1)(dense2)

    # Создаем модель
    model = Model(inputs=[area_input, floor_input, total_floors_input, year_built_input, condition_input, district_input], outputs=output)

    # Компиляция модели
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# Определяем количество уникальных районов и размер эмбеддинга
num_districts = len(data['district'].unique())
embedding_dim = 5

# Создаем модель
model = create_model(num_districts, embedding_dim)

# Обучаем модель
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=8,
    validation_data=(X_test, y_test)
)

# Оцениваем модель на тренировочных данных
y_train_pred = model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

print(f"Train MAE: {train_mae:.2f}, Train MAPE: {train_mape:.2%}")

# Оцениваем модель на тестовых данных
y_test_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"MAE: {mae:.2f}, MAPE: {mape:.2%}")

# Извлечение эмбеддингов для районов
embedding_layer = model.get_layer('embedding')
embedding_weights = embedding_layer.get_weights()[0]

# Сохранение эмбеддингов в DataFrame
embeddings_df = pd.DataFrame(embedding_weights, index=data['district'].unique())

# Вывод эмбеддингов
print(embeddings_df.head())

In [ ]:
from catboost import CatBoostRegressor

# Создаем DataFrame с эмбеддингами районов
X_train_embeddings = pd.concat([pd.DataFrame(X_train), embeddings_df.loc[X_train['district']].reset_index(drop=True)], axis=1)
X_test_embeddings = pd.concat([pd.DataFrame(X_test), embeddings_df.loc[X_test['district']].reset_index(drop=True)], axis=1)

# Удаляем лишний столбец district
X_train_embeddings = X_train_embeddings.drop(columns=['district'])
X_test_embeddings = X_test_embeddings.drop(columns=['district'])

# Создаем и обучаем модель CatBoost
catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=100)

catboost_model.fit(
    X_train_embeddings,
    y_train,
    eval_set=(X_test_embeddings, y_test),
    early_stopping_rounds=100
)

# Оцениваем модель на тестовых данных
y_test_pred = catboost_model.predict(X_test_embeddings)
catboost_mae = mean_absolute_error(y_test, y_test_pred)
catboost_mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"CatBoost MAE: {catboost_mae:.2f}, CatBoost MAPE: {catboost_mape:.2%}")

1. Зачем учитывать остальные факторы при кодировании района?
Когда мы используем эмбеддинги для категориального признака "район" и одновременно учитываем остальные факторы (площадь, этаж, этажность, год постройки, отделка), мы помогаем модели лучше понять, как эти факторы взаимодействуют с районом. Это дает следующие преимущества:

Учет взаимодействий : Эмбеддинги позволяют модели выявить и использовать взаимодействия между районом и другими признаками. Например, влияние района на цену может зависеть от этажности здания или состояния отделки.
Улучшенная обобщающая способность : Модель может лучше обобщать, когда она учитывает все признаки сразу, так как она видит общую картину данных.

2. Можно ли закодировать район без учета других факторов и цены?  
Да, можно закодировать район с использованием эмбеддингов без учета других факторов и цены. Однако это может быть менее эффективным, так как модель не сможет учитывать взаимодействия между районом и другими признаками. Если вы хотите использовать эмбеддинги только для района, то модель будет обучаться на основе только этого признака, что может ограничить её способность делать точные предсказания.

1. Обучение нейросети : Мы обучаем нейросеть для получения эмбеддингов районов.
2. Извлечение эмбеддингов : Мы извлекаем эмбеддинги районов и сохраняем их в DataFrame.
3. Создание новых признаков : Мы создаем новые признаки, объединяя исходные признаки с эмбеддингами районов.
4. Обучение бустинговой модели : Мы обучаем бустинговую модель (в данном случае CatBoost) на новых признаках, включая эмбеддинги районов.
5. Оценка модели : Мы оцениваем модель на тестовых данных и выводим метрики MAE и MAPE.